<a href="https://colab.research.google.com/github/peculab/PROGRAM/blob/main/hw6_%E8%AA%B2%E8%A1%A8%E6%9F%A5%E8%A9%A2%E8%88%87%E6%8F%90%E9%86%92%E7%B4%A0%E6%9D%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [12]:
from google.colab import files
import os

print("請上傳您的 PDF 課表檔案...")
# 顯示上傳按鈕
uploaded = files.upload()

if not uploaded:
  print("\n⚠️ 您沒有上傳任何檔案。")
else:
  # 取得上傳檔案的名稱
  pdf_filename = next(iter(uploaded))
  print(f"\n✅ 成功上傳檔案: '{pdf_filename}'")

  # 檢查是否為 PDF (簡易檢查)
  if not pdf_filename.lower().endswith('.pdf'):
    print(f"⚠️ 警告：您上傳的檔案 '{pdf_filename}' 不是 PDF 檔。")

請上傳您的 PDF 課表檔案...


Saving export.pdf to export.pdf

✅ 成功上傳檔案: 'export.pdf'


In [14]:
%store pdf_filename

Stored 'pdf_filename' (str)


In [13]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

In [16]:
import time

# 讀取上一個儲存格存的檔案名稱
try:
    %store -r pdf_filename
    if not os.path.exists(pdf_filename):
        print(f"❌ 找不到檔案 '{pdf_filename}'，請重新執行上一個儲存格上傳。")
    else:
        print(f"準備處理檔案: '{pdf_filename}'")

        # 1. 上傳檔案到 Gemini File API
        print("正在上傳 PDF 至 Gemini File API...")
        gemini_file = genai.upload_file(path=pdf_filename,
                                        display_name="114-1 課程表 PDF")
        print(f"✅ 檔案上傳成功！(File URI: {gemini_file.uri})")

        # 2. 準備 Prompt
        # 這就是您第一次給我的指示，我把它變成了給 Gemini 的 Prompt
        prompt_text = f"""
        你是一位 helpful AI assistant。
        請分析我上傳的 PDF 課表檔案 (檔案名稱: {pdf_filename})。

        這份 PDF 課表是「國立臺灣師範大學114學年度第1學期課表」。
        授課教師是「蔡芸琤」。

        請幫我將這份課表，轉換成每周的紀錄。
        學期開學日是 2025/9/1，學期結束日是 2025/12/19。

        你的輸出**必須**是 CSV 格式，並且**只**輸出 CSV 內容 (包含標頭)，
        不要有任何其他說明文字 (例如 "好的，這是..." 或 "```csv" 標記)。

        CSV 格式的欄位必須如下：
        "日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"

        - 「攜帶品」、「先讀章節」、「備註」這三欄因為 PDF 中沒有，請保持空白。
        - 「日期」範圍請涵蓋 2025/9/1 到 2025/12/19 之間所有有課的日期。
        - 請仔細解析 PDF 中的表格，找出所有課程、時間和地點。
        """

        # 3. 呼叫 Gemini 1.5 Pro 模型
        print("正在向 Gemini 1.5 Pro 提出請求 (這可能需要一點時間)...")
        model = genai.GenerativeModel('gemini-2.5-pro')

        # 發送包含「Prompt文字」和「上傳的檔案」的請求
        response = model.generate_content([prompt_text, gemini_file])

        # 4. 刪除在 Gemini 雲端上的檔案 (避免佔用空間)
        print("正在從 Gemini 雲端刪除暫存檔案...")
        genai.delete_file(gemini_file.name)
        print("✅ 暫存檔案已刪除。")

        # 5. 顯示結果並儲存
        csv_output = response.text.strip().replace("```csv\n", "").replace("```", "")
        print("\n--- 🤖 Gemini 產生的 CSV 內容 ---")
        print(csv_output)
        print("---------------------------------")

        # 6. 將 Gemini 產生的 CSV 內容存成一個檔案
        with open("gemini_schedule.csv", "w", encoding='utf-8') as f:
            f.write(csv_output)

        print("\n✅ 已將 Gemini 產生的結果儲存為 'gemini_schedule.csv'")

except NameError:
    print("❌ 找不到 'pdf_filename'。請先執行上一個儲存格來上傳檔案。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")
    # 如果發生錯誤，也嘗試刪除已上傳的檔案
    try:
        if 'gemini_file' in locals():
            genai.delete_file(gemini_file.name)
            print("已清理遠端暫存檔案。")
    except Exception as del_e:
        print(f"清理檔案時發生錯誤: {del_e}")

準備處理檔案: 'export.pdf'
正在上傳 PDF 至 Gemini File API...
✅ 檔案上傳成功！(File URI: https://generativelanguage.googleapis.com/v1beta/files/k8q0zspbti1x)
正在向 Gemini 1.5 Pro 提出請求 (這可能需要一點時間)...
正在從 Gemini 雲端刪除暫存檔案...
✅ 暫存檔案已刪除。

--- 🤖 Gemini 產生的 CSV 內容 ---
"日期","星期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"
"2025/09/01","星期一","資料庫系統","09:10","12:10","TB311教室","","",""
"2025/09/02","星期二","網際網路概論","09:10","12:10","TB311教室","","",""
"2025/09/03","星期三","資料探勘在教育上的應用","14:20","17:20","TB312教室","","",""
"2025/09/04","星期四","程式語言","09:10","12:10","TB311教室","","",""
"2025/09/05","星期五","專題製作(二) D","20:30","22:15","TB212教室","","",""
"2025/09/08","星期一","資料庫系統","09:10","12:10","TB311教室","","",""
"2025/09/09","星期二","網際網路概論","09:10","12:10","TB311教室","","",""
"2025/09/10","星期三","資料探勘在教育上的應用","14:20","17:20","TB312教室","","",""
"2025/09/11","星期四","程式語言","09:10","12:10","TB311教室","","",""
"2025/09/12","星期五","專題製作(二) D","20:30","22:15","TB212教室","","",""
"2025/09/15","星期一","資料庫系統","09:10","12:10","TB311教室","","",""


In [18]:
import pandas as pd
import gspread

# --- 1. 定義您的 Google Sheet 資訊 (您已提供) ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
WORKSHEET_NAME = "HW6"

# --- 2. 讀取 Gemini 產生的 CSV 檔案 ---
# (假設 'gemini_schedule.csv' 檔案在 Colab 的根目錄下)
try:
    df_to_upload = pd.read_csv("gemini_schedule.csv")
    print(f"✅ 成功讀取 'gemini_schedule.csv'，共 {len(df_to_upload)} 筆資料。")

    # --- 3. 準備 gspread (假設 gc 已經過授權) ---
    # gc = gspread.authorize(creds) # 這行在您之前的儲存格應該已經執行過了
    df_to_upload = df_to_upload.fillna('')

    # --- 4. 開啟 Google Sheet 和指定的分頁 ---
    print(f"正在開啟 Google Sheet (URL: {SHEET_URL})...")
    gsheets = gc.open_by_url(SHEET_URL)

    print(f"正在存取分頁: '{WORKSHEET_NAME}'...")
    try:
        # 嘗試取得分頁，如果不存在就建立一個新的
        worksheet = gsheets.worksheet(WORKSHEET_NAME)
    except gspread.exceptions.WorksheetNotFound:
        print(f"找不到分頁 '{WORKSHEET_NAME}'，將自動建立新分頁...")
        worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

    # --- 5. 將 DataFrame 轉換為 list (包含標頭) ---
    data_to_write = [df_to_upload.columns.values.tolist()] + df_to_upload.values.tolist()

    # --- 6. 清空分頁並寫入新資料 ---
    print("正在清空分頁內容...")
    worksheet.clear()

    print(f"正在將 {len(data_to_write) - 1} 筆資料寫入 '{WORKSHEET_NAME}' 分頁...")
    worksheet.update(data_to_write)

    print(f"\n✅ 成功將 'gemini_schedule.csv' 的內容寫入至 '{WORKSHEET_NAME}' 分頁！")

except FileNotFoundError:
    print("❌ 錯誤：找不到 'gemini_schedule.csv' 檔案。")
    print("請確認您執行 Gemini API 的儲存格已成功產生這個檔案。")
except pd.errors.EmptyDataError:
    print("❌ 錯誤：'gemini_schedule.csv' 是空的，沒有資料可以上傳。")
except Exception as e:
    print(f"❌ 執行時發生錯誤：{e}")

✅ 成功讀取 'gemini_schedule.csv'，共 80 筆資料。
正在開啟 Google Sheet (URL: https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing)...
正在存取分頁: 'HW6'...
正在清空分頁內容...
正在將 80 筆資料寫入 'HW6' 分頁...

✅ 成功將 'gemini_schedule.csv' 的內容寫入至 'HW6' 分頁！


In [5]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
WORKSHEET_NAME = "HW6"
TIMEZONE = "Asia/Taipei"

In [6]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,日期,星期,課程名稱,時間(起),時間(迄),地點,攜帶品,先讀章節,備註
0,2025-11-03,一,資料庫系統,9:10,12:00,誠101,"筆電, 充電器",教科書 Ch.6 SQL,
1,2025-11-03,一,生成式AI實務,13:10,16:00,科201,"耳機, 筆電",Lecture Notes Week 6,
2,2025-11-04,二,資料庫系統,9:10,12:00,誠101,"筆電, 充電器",教科書 Ch.6 SQL,
3,2025-11-04,二,生成式AI實務,13:10,16:00,科201,"耳機, 筆電",Lecture Notes Week 6,
4,2025-11-05,三,資料庫系統,9:10,12:00,誠101,"筆電, 充電器",教科書 Ch.6 SQL,


In [7]:
import pytz
from IPython.display import display, Markdown
import datetime as dt

# === 參數：指定要查的星期 ===
# 可填 "一","二","三","四","五","六","日"，或用英文 "Mon" ... "Sun"
TARGET_WEEKDAY = "三"  # 例：查星期三的課

# === 小工具函式 ===
TW_TZ = pytz.timezone(TIMEZONE)

week_map_tw = {"一":0,"二":1,"三":2,"四":3,"五":4,"六":5,"日":6}
week_map_en = {"Mon":0,"Tue":1,"Wed":2,"Thu":3,"Fri":4,"Sat":5,"Sun":6}

def normalize_weekday(w):
    if w in week_map_tw: return week_map_tw[w]
    if w in week_map_en: return week_map_en[w]
    raise ValueError("星期請用：一~日 或 Mon~Sun")

def ensure_datetime(s):
    try:
        return pd.to_datetime(s).date()
    except Exception:
        return pd.NaT

def week_monday(any_date):
    # 回到本週一
    return any_date - dt.timedelta(days=any_date.weekday())

def date_range_this_week(today=None):
    # 取得「本週一 ~ 本週日」日期區間（以台北時區為準）
    now = dt.datetime.now(TW_TZ).date() if today is None else today
    mon = week_monday(now)
    sun = mon + dt.timedelta(days=6)
    return mon, sun

def summarize_courses(day_df):
    if day_df.empty: return "本日無課"
    parts = []
    for _, r in day_df.iterrows():
        parts.append(f"{r['時間(起)']}-{r['時間(迄)']} {r['課程名稱']}（{r['地點']}）")
    return "；".join(parts)

# === 過濾指定星期的課程（自動只抓「本週」） ===
df["日期"] = df["日期"].apply(ensure_datetime)
df = df.dropna(subset=["日期"])

target_wd = normalize_weekday(TARGET_WEEKDAY)
this_mon, this_sun = date_range_this_week()

mask_week = (df["日期"] >= this_mon) & (df["日期"] <= this_sun)
mask_wd   = df["日期"].apply(lambda d: d.weekday()==target_wd)
today_df  = df[mask_week & mask_wd].copy().sort_values(["日期","時間(起)"])

# === 顯示查詢結果（指定星期的所有課） ===
display(Markdown(f"### 本週（{this_mon} ~ {this_sun}）星期{TARGET_WEEKDAY}課程"))
if today_df.empty:
    display(Markdown("> 本日無課"))
else:
    display(today_df[["日期","課程名稱","時間(起)","時間(迄)","地點","攜帶品","先讀章節","備註"]])

# === 產生一句「行前提醒」 ===
def make_one_line_reminder(rows):
    # ── 方案A：不接模型的簡單規則版（免 API Key）
    if rows.empty:
        return "本日無課，不需準備。"
    items = []
    readings = []
    for _, r in rows.iterrows():
        if str(r.get("攜帶品","")).strip():
            items.append(str(r["攜帶品"]).strip())
        if str(r.get("先讀章節","")).strip():
            readings.append(str(r["先讀章節"]).strip())
    items_txt = "；".join(dict.fromkeys(items)) if items else "一般上課用品"
    read_txt  = "；".join(dict.fromkeys(readings)) if readings else "無指定預習"
    return f"請攜帶：{items_txt}；預習：{read_txt}。"

one_line = make_one_line_reminder(today_df)
display(Markdown("#### 行前提醒（一句話）"))
display(Markdown(f"> {one_line}"))

# === 回寫「本週重點」分頁 ===
# 若該分頁不存在就自動建立
SHEET_WEEKLY = "本週重點"
try:
    wk = gc.open_by_url(SHEET_URL).worksheet(SHEET_WEEKLY)
except gspread.exceptions.WorksheetNotFound:
    wk = gc.open_by_url(SHEET_URL).add_worksheet(title=SHEET_WEEKLY, rows=100, cols=20)

# 組成回寫資料：可以一週寫一列
summary_text = summarize_courses(df[mask_week].sort_values(["日期","時間(起)"]))
now_str = dt.datetime.now(TW_TZ).strftime("%Y-%m-%d %H:%M")

row = {
    "週次起訖": f"{this_mon} ~ {this_sun}",
    "查詢星期": f"星期{TARGET_WEEKDAY}",
    "當日課程列表": summarize_courses(today_df),
    "行前提醒一句話": one_line,
    "更新時間": now_str
}

# 追加寫入（保留歷史）
existing = pd.DataFrame(wk.get_all_records())
outdf = pd.concat([existing, pd.DataFrame([row])], ignore_index=True) if not existing.empty else pd.DataFrame([row])

wk.clear()
set_with_dataframe(wk, outdf)

display(Markdown("✅ 已回寫到 **本週重點** 分頁（保留歷史列）。"))


### 本週（2025-11-03 ~ 2025-11-09）星期三課程

,日期,課程名稱,時間(起),時間(迄),地點,攜帶品,先讀章節,備註
5,2025-11-05,生成式AI實務,13:10,16:00,科201,"耳機, 筆電",Lecture Notes Week 6,
4,2025-11-05,資料庫系統,9:10,12:00,誠101,"筆電, 充電器",教科書 Ch.6 SQL,


#### 行前提醒（一句話）

> 請攜帶：耳機, 筆電；筆電, 充電器；預習：Lecture Notes Week 6；教科書 Ch.6 SQL。

✅ 已回寫到 **本週重點** 分頁（保留歷史列）。

In [10]:
# === 加入 Gradio 編輯面板 ===
import gradio as gr

def save_edited_reminder(edited_text):
    """讓使用者編輯後回存到 Google Sheet"""
    from datetime import datetime
    import pytz
    tz = pytz.timezone(TIMEZONE)
    now_str = datetime.now(tz).strftime("%Y-%m-%d %H:%M")

    # 重新讀取目前的分頁資料
    existing = pd.DataFrame(wk.get_all_records())
    # 追加更新列
    row = {
        "週次起訖": f"{this_mon} ~ {this_sun}",
        "查詢星期": f"星期{TARGET_WEEKDAY}",
        "當日課程列表": summarize_courses(today_df),
        "行前提醒一句話": edited_text,
        "更新時間": now_str
    }
    outdf = pd.concat([existing, pd.DataFrame([row])], ignore_index=True)
    wk.clear()
    set_with_dataframe(wk, outdf)
    return f"✅ 已更新至 Google Sheet『本週重點』分頁，目前共 {len(outdf)} 筆。"

# Gradio 介面
with gr.Blocks(title="行前提醒編輯器") as demo:
    gr.Markdown("## ✏️ 行前提醒（可編輯並回存至 Google Sheet）")
    gr.Markdown("這裡會顯示模型或程式預設生成的行前提醒，使用者可以修改後按下『提交』回存。")
    reminder_box = gr.Textbox(label="行前提醒一句話", value=one_line, lines=3)
    save_btn = gr.Button("💾 提交並回存")
    status = gr.Markdown()

    save_btn.click(save_edited_reminder, inputs=[reminder_box], outputs=[status])

demo.launch(share=True, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8fe25a0af872c1c805.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
